In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import fetch_openml

# Загрузка датасета steel-plates-fault
dataset = fetch_openml('steel-plates-fault', version=1)

# Разделение данных на признаки и целевую переменную
X = dataset.data
y = dataset.target

# Преобразование целевой переменной в числовой формат
y = y.astype(float)
y = (y - y.min()) / (y.max() - y.min())  # Normalize y to [0, 1]

# Создание целевой переменной для обучения и тестирования
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42, shuffle=False)

# Получение соответствующих X для y_train и y_test
X_train = X[:len(y_train)]
X_test = X[len(X_train):]

# Преобразование данных в PyTorch формат
X_train = torch.FloatTensor(X_train.to_numpy())
X_test = torch.FloatTensor(X_test.to_numpy())

# Создание целевой переменной для обучения и тестирования в PyTorch формате
y_train_tensor = torch.FloatTensor(y_train.values)
y_test_tensor = torch.FloatTensor(y_test.values)

# Создание модели PyTorch
class SteelPlatesFaultModel(nn.Module):
    def __init__(self):
        super(SteelPlatesFaultModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.sigmoid(x)

model = SteelPlatesFaultModel()

# Оптимизатор
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 100
predictions = []
test_losses = []
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Убедимся, что размерности совпадают
    assert X_train.shape[0] == y_train_tensor.shape[0], f"X_train shape: {X_train.shape}, y_train_tensor shape: {y_train_tensor.shape}"

    outputs = model(X_train)
    loss = nn.BCELoss()(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_loss = nn.BCELoss()(model(X_test), y_test_tensor.unsqueeze(1))
        predictions.append((model(X_test).squeeze()).numpy())
        test_losses.append(test_loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_loss.item():.4f}')

# Оценка модели на тестовых данных
y_pred = torch.FloatTensor(predictions)
print("\nClassification Report:")
if len(y_test) == len(y_pred):
    print(classification_report(y_test.values, y_pred.numpy()))
else:
    print("Warning: Inconsistent number of samples in y_test and predictions.")
    print(f"y_test shape: {y_test.shape}")
    print(f"predictions shape: {y_pred.shape}")
    print("Please check your model's performance or data preprocessing.")

# Визуализация обучения модели
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(range(num_epochs), test_losses)
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

# Additional checks
print(f"Number of samples in y_test: {len(y_test)}")
print(f"Number of predictions made by the model: {len(predictions)}")
if len(y_test) != len(predictions):
    print("Warning: Number of samples in y_test does not match the number of predictions.")
    print("This could indicate an issue with model training or data handling.")

Описание кода:

1. Импорт необходимых библиотек:

      1. PyTorch для создания и обучения модели

      2. scikit-learn для разделения данных на обучающую и тестовую выборки

      3. NumPy для числовых операций

      4. Метрики sklearn для оценки качества модели
2. Загрузка набора данных "steel-plates-fault" с помощью fetch_openml()
3. Преобразование целевой переменной y в диапазон [0, 1]
4. Разделение данных на обучающую и тестовую выборки. Используется параметр shuffle=False, чтобы сохранить исходную последовательность данных.
5. Получение соответствующих X для y_train и y_test
6. Преобразование данных в формат PyTorch
7. Создание модели PyTorch с тремя полными слоями
8. Определение оптимизатора и обучения модели
9. Оценка качества модели с помощью метрик sklearn и визуализация потерь на каждом эпохе.






Этот код представляет собой нейронную сеть для классификации данных из набора данных "steel-plates-fault".